In [11]:
import sqlite3
import pandas as pd
import numpy as np

def calculate_fantasy_points(row):
    """Calculate fantasy points per game for a player based on league scoring rules."""
    fantasy_points = (
        row['AST'] * 2 +
        row['BLK'] * 4 +
        row['FGA'] * -1 +
        row['FG'] * 2 +
        row['FTA'] * -1 +
        row['FT'] * 1 +
        row['PTS'] * 1 +
        row['TRB'] * 1 +
        row['STL'] * 4 +
        row['3P'] * 1 +
        row['TOV'] * -2
    )
    return fantasy_points

def calculate_fantasy_points_98_23(db_path='data/player_data.db'):
    """Calculate Fantasy Points Per Game (FP/G) for each player's season from 98 to 23."""
    conn = sqlite3.connect(db_path)
    
    # Load the 98-23 stats data into a DataFrame
    query = "SELECT * FROM players_98_23"
    players_98_23 = pd.read_sql_query(query, conn)
    
    # Calculate fantasy points per game
    players_98_23['FP/G'] = players_98_23.apply(calculate_fantasy_points, axis=1)
    
    # Create a new table with the calculated fantasy points
    players_98_23.to_sql('players_98_23_fantasy', conn, if_exists='replace', index=False)
    
    conn.close()
    print("Fantasy Points Per Game (FP/G) calculated and added to new table 'players_98_23_fantasy'.")

    return players_98_23

def analyze_fantasy_points(df):
    """Perform basic analysis on the calculated fantasy points."""
    print("\nFantasy Points Analysis:")
    print(f"Average FP/G: {df['FP/G'].mean():.2f}")
    print(f"Median FP/G: {df['FP/G'].median():.2f}")
    print(f"Max FP/G: {df['FP/G'].max():.2f}")
    print(f"Min FP/G: {df['FP/G'].min():.2f}")
    
    print("\nTop 10 Fantasy Performers:")
    top_10 = df.sort_values('FP/G', ascending=False).head(10)[['Player', 'Season', 'Pos', 'FP/G']]
    print(top_10)
    
    print("\nAverage FP/G by Position:")
    avg_by_pos = df.groupby('Pos')['FP/G'].mean().sort_values(ascending=False)
    print(avg_by_pos)

    # Add analysis for LeBron's 2008-09 season
    lebron_2009 = df[(df['Player'] == 'LeBron James') & (df['Season'] == '2008-09')]
    if not lebron_2009.empty:
        print("\nLeBron James 2008-09 Season:")
        print(lebron_2009[['Player', 'Season', 'FP/G']])

if __name__ == "__main__":
    df_with_fantasy_points = calculate_fantasy_points_98_23()
    analyze_fantasy_points(df_with_fantasy_points)

Fantasy Points Per Game (FP/G) calculated and added to new table 'players_98_23_fantasy'.

Fantasy Points Analysis:
Average FP/G: 16.06
Median FP/G: 14.00
Max FP/G: 61.70
Min FP/G: -8.00

Top 10 Fantasy Performers:
                  Player    Season Pos  FP/G
14377       Nikola Jokić   2021-22   C  61.7
15116       Nikola Jokić  2022-010   C  60.4
12192       James Harden   2018-19  PG  59.1
13625       Nikola Jokić   2020-21   C  58.6
12888       James Harden   2019-20  SG  58.0
12083      Anthony Davis   2018-19   C  57.7
11221  Russell Westbrook   2016-17  PG  57.6
1390   Shaquille O'Neal*   1999-00   C  57.5
14958        Luka Dončić  2022-010  PG  57.3
6781        LeBron James   2009-10  SF  57.2

Average FP/G by Position:
Pos
PG-SF       25.000000
SG-PF       22.350000
SF-SG       18.390625
PG          17.509479
SG-PG       17.203846
PG-SG       16.665517
PF          15.830089
C           15.811046
SF          15.625851
SG          15.541727
SF-PF       14.756522
PF-C        14.68

In [19]:
from data.setup_db import load_csv_to_db

In [18]:
def get_fantasy_points_history(conn):
    """
    Create a dataframe of current fantasy players averaging over 20 FP/G
    with their fantasy points per game for each year, handling traded players.
    """
    # Get current players averaging over 20 FP/G
    query1 = """
    SELECT player, fantasy_team, fp_per_g as current_fp_per_g
    FROM player_fantasy_stats
    WHERE fp_per_g > 20
    """
    current_players = pd.read_sql_query(query1, conn)

    # Escape single quotes in player names and create the list
    players_list = ', '.join(f"'{player.replace("'", "''")}'" for player in current_players['player'])

    # Get historical data for these players
    query2 = f"""
    SELECT Player, Season, "FP/G" as fp_per_g, G as games_played
    FROM players_98_23_fantasy
    WHERE Player IN ({players_list})
    """
    historical_stats = pd.read_sql_query(query2, conn)

    # Aggregate stats for players with multiple entries in a season
    historical_stats['weighted_fp'] = historical_stats['fp_per_g'] * historical_stats['games_played']
    aggregated_stats = historical_stats.groupby(['Player', 'Season']).agg({
        'weighted_fp': 'sum',
        'games_played': 'sum'
    }).reset_index()
    aggregated_stats['fp_per_g'] = aggregated_stats['weighted_fp'] / aggregated_stats['games_played']

    # Pivot the aggregated stats to create columns for each season
    pivoted_stats = aggregated_stats.pivot(index='Player', columns='Season', values='fp_per_g')
    
    # Merge with current fantasy team information
    result = current_players.merge(pivoted_stats, left_on='player', right_index=True, how='left')

    # Sort by current FP/G
    result = result.sort_values("current_fp_per_g", ascending=False)

    return result

# In your main execution block:
if __name__ == "__main__":
    conn = sqlite3.connect('data/player_data.db')
    
    # Your existing code here...

    # Add this new section
    print("\nFantasy Points per Game History for Current High Performers:")
    fantasy_points_history = get_fantasy_points_history(conn)
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.width', None)  # Don't wrap to multiple lines
    print(fantasy_points_history)

    fantasy_points_history.to_csv('data/processed/fpts_since_1998.csv')
    conn.close()


Fantasy Points per Game History for Current High Performers:
                    player fantasy_team  current_fp_per_g  1997-98  1998-99  \
1              Luka Doncic          $¢$             66.34      NaN      NaN   
0             Nikola Jokic          CCC             65.24      NaN      NaN   
57             Joel Embiid     Jmarr237             63.72      NaN      NaN   
2    Giannis Antetokounmpo          BBB             60.19      NaN      NaN   
4            Anthony Davis     Jmarr237             57.08      NaN      NaN   
..                     ...          ...               ...      ...      ...   
175    Dorian Finney-Smith        Orcas             20.47      NaN      NaN   
179      Simone Fontecchio        Orcas             20.45      NaN      NaN   
145        Harrison Barnes      Teacups             20.33      NaN      NaN   
174            Isaac Okoro     Jmarr237             20.33      NaN      NaN   
184          Dyson Daniels         SERP             20.20      NaN   

In [17]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

def get_player_data(conn):
    """Retrieve player data including age and fantasy points for all seasons."""
    query = """
    SELECT p.Player, p.Season, p."FP/G" as fp_per_g, p.Age, p.G as games_played
    FROM players_98_23_fantasy p
    """
    return pd.read_sql_query(query, conn)

def prepare_data(df):
    """Prepare data for KNN analysis, handling duplicate entries."""
    # Aggregate data for players with multiple entries in a season
    df['weighted_fp'] = df['fp_per_g'] * df['games_played']
    aggregated = df.groupby(['Player', 'Age', 'Season']).agg({
        'weighted_fp': 'sum',
        'games_played': 'sum'
    }).reset_index()
    aggregated['fp_per_g'] = aggregated['weighted_fp'] / aggregated['games_played']

    # Pivot the aggregated data to have seasons as columns
    pivoted = aggregated.pivot(index=['Player', 'Age'], columns='Season', values='fp_per_g').reset_index()
    
    # Calculate career average up to each age
    for age in range(19, 41):  # Assuming players are between 19 and 40
        age_cols = [col for col in pivoted.columns if col not in ['Player', 'Age'] and pivoted[pivoted['Age'] == age][col].notna().any()]
        pivoted[f'avg_to_{age}'] = pivoted[age_cols].mean(axis=1)
    
    return pivoted

def project_player(player_data, current_players, k=4):
    """Project future performance for current players using KNN."""
    scaler = StandardScaler()
    
    # Prepare features for KNN
    features = ['Age'] + [col for col in player_data.columns if col.startswith('avg_to_')]
    X = scaler.fit_transform(player_data[features])
    
    # Fit KNN model
    knn = NearestNeighbors(n_neighbors=k+1, metric='euclidean')
    knn.fit(X)
    
    projections = []
    
    for _, player in current_players.iterrows():
        age = player['Age']
        fp_per_g = player['current_fp_per_g']
        
        # Find the average up to the current age
        avg_col = f'avg_to_{age}'
        if avg_col not in features:
            avg_col = features[-1]  # Use the last available average if age is out of range
        
        # Prepare player data for KNN
        player_features = np.array([[age, fp_per_g] + [fp_per_g] * (len(features) - 2)])
        player_scaled = scaler.transform(player_features)
        
        # Find K nearest neighbors
        distances, indices = knn.kneighbors(player_scaled)
        
        # Get the neighbors' data
        neighbors = player_data.iloc[indices[0][1:]]  # Exclude the first neighbor (itself)
        
        # Project next 5 years
        projection = [player['player'], player['fantasy_team'], age, fp_per_g]
        for i in range(1, 6):
            projected_age = age + i
            if projected_age > 40:
                projection.append(np.nan)
            else:
                projected_avg = neighbors[neighbors['Age'] == projected_age]['fp_per_g'].mean()
                projection.append(projected_avg)
        
        projections.append(projection)
    
    # Create DataFrame with projections
    columns = ['player', 'fantasy_team', 'current_age', 'current_fp_per_g'] + [f'projected_year_{i}' for i in range(1, 6)]
    return pd.DataFrame(projections, columns=columns)

def get_column_names(conn, table_name):
    """Get column names from a given table."""
    query = f"PRAGMA table_info({table_name})"
    cursor = conn.cursor()
    cursor.execute(query)
    return [row[1] for row in cursor.fetchall()]

def main():
    conn = sqlite3.connect('data/player_data.db')
    
    # Get column names from tables
    fantasy_stats_columns = get_column_names(conn, 'player_fantasy_stats')
    players_98_23_columns = get_column_names(conn, 'players_98_23_fantasy')
    
    # Determine the correct column name for fantasy points per game
    fp_per_g_col = 'fp_per_g' if 'fp_per_g' in fantasy_stats_columns else 'FP/G'
    
    # Get current players data, joining with players_98_23_fantasy to get Age
    current_players_query = f"""
    SELECT pfs.player, pfs.fantasy_team, pfs."{fp_per_g_col}" as current_fp_per_g, 
           p98.Age as Age
    FROM player_fantasy_stats pfs
    JOIN (
        SELECT Player, Age
        FROM players_98_23_fantasy
        WHERE Season = (SELECT MAX(Season) FROM players_98_23_fantasy)
    ) p98 ON pfs.player = p98.Player
    WHERE pfs."{fp_per_g_col}" > 20
    """
    current_players = pd.read_sql_query(current_players_query, conn)
    
    # Get historical player data
    player_data = get_player_data(conn)
    prepared_data = prepare_data(player_data)
    
    # Project future performance
    projections = project_player(prepared_data, current_players)
    
    # Sort by current fantasy points per game
    projections = projections.sort_values('current_fp_per_g', ascending=False)
    
    print("Projected Fantasy Points per Game for the Next 5 Years:")
    print(projections)
    
    # Save projections to CSV
    projections.to_csv('fantasy_points_projections.csv', index=False)
    print("\nProjections saved to 'fantasy_points_projections.csv'")
    
    conn.close()

if __name__ == "__main__":
    main()

/Users/maxwellpatterson/opt/anaconda3/envs/zenv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


KeyError: 'fp_per_g'